In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size,output_size):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        x = x.view(x.size(0),-1)  #将多维度的x展开成一维，以链接全连接层
        out = self.layers(x)
        return out

In [ ]:
input_size = 784   #784为像素总数
hidden_size = 128
output_size = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 30

In [ ]:
transform = transforms.Compose([             #用Compose将多个预处理步骤整合在一起
transforms.ToTensor(),                        #transforms.ToTensor()可以将PIL和numpy格式的数据从[0,255]范围转换到[0,1] 
#具体做法其实就是将原始数据除以255。另外原始数据的shape是（H x W x C），通过transforms.ToTensor()后shape会变为（C x H x W）         
transforms.Normalize((0.1307,),(0.3081,))           #output[channel] = (input[channel] - mean[channel]) / std[channel]。
#这里[channel]的意思是指对特征图的每个通道都进行这样的操作。【mean为均值，std为标准差】
])

train_dataset = datasets.MNIST(root = './data', train = True, transform=transform, download=True)   #手写数据集
test_dataset = datasets.MNIST(root = './data', train = False, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#  test测试集，train训练集，训练集用于迭代训练（相当于模拟考提升），测试集用于最终测试（相当于最终考试）
model = MLP(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()    #交叉熵损失函数，一种损失函数
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
correction = []
correction_ = []
Loss = []

In [ ]:
for epoch in range(0, num_epochs):
    model.train()
    loss_ = 0
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):  #enumerate返回索引和对应值
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_ += loss.item()
        
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)           
        total += labels.size(0)            
        correct += (predicted == labels).sum().item()



    Loss.append(loss_)    
    model.eval()        #打开评估模式
    total_ = 0
    correct_ = 0

        
    for images, labels in test_loader:
        outputs_ = model(images)
        _,predicted_ = torch.max(outputs_.data, 1)        #torch.max(input, dim),dim是max函数索引的维度0/1，0是每列的最大值，1是每行的最大值
        #函数会返回两个tensor，第一个tensor是每行的最大值，softmax的输出中最大的是1，所以第一个tensor是全1的tensor；第二个tensor是每行最大值的索引。
        total_ += labels.size(0)     #计算lables总量        
        #torrch.size(0)中的0表示第0维度的数据数量,如a = torch.tensor([[1,2,3], [4,5,6]])，a.size（0）为2，即有[1,2,3]，[4,5,6]两个数据
        correct_ += (predicted_ == labels).sum().item()
        #(a == b).sum()，若a,b为张量，返回tensor(1)。若为numpy，则返回1。.item()用于取出tensor中的值。
    a = float(100 * correct / total)
    b = float(100 * correct_ / total_)
    correction.append(a)
    correction_.append(b)
x = [i for i in range(1,num_epochs+1)]
plt.figure(figsize=(15,15))
plt.subplot(3,3,1)
plt.title('Zetao_Li')
plt.xlabel('epoch')
plt.ylabel('train_correct')
plt.plot(x,correction)
plt.subplot(3,3,2)
plt.title('Zetao_Li')
plt.xlabel('epoch')
plt.ylabel('test_correct')
plt.plot(x,correction_)
plt.subplot(3,3,3)
plt.title('Zetao_Li')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.plot(x,Loss)
plt.show()


    


